# **Модуль Б.  Разведочный анализ данных**

***Перед началом работы нужно импортировать необходимые библиотеки.***

In [1]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

### *2.3 Кластеризация набора данных*
Выберите модель кластеризации данных. Задача кластеризации – сегментация поездок, определение схожих групп объектов в каждом наборе данных. В результате должно быть выявлено 3 кластера: в первый кластер попадают все маршруты, для которых поездка в заданное время оптимальна, во второй кластер попадают маршруты с нежелательным временем поездки, в третий кластер попадают маршруты, от поездок по которым следует воздержаться в заданный период времени.

***Чтобы начать кластеризацию, первым делом нужно определиться, куда сохранять кластеры, чтобы в будущем могли воспользоваться ими для других анализов. Для эффективности стоит сохранять в таблице БД. Когда решились, нужно написать функцию, которая создаёт таблицу для хранения результатов кластеризации.***

In [2]:
def create_cluster_table(db_path):
    """
    Функция создаёт таблицу с заданной структурой, если её в БД нету
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS trip_clusters (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            trip_distance REAL,
            trip_duration REAL,
            cluster_kmeans TEXT,
            cluster_dbscan TEXT,
            cluster_hierarchical TEXT
        )
    """)
    
    conn.commit()
    conn.close()

***По заданию должно быть выявлено 3 кластера:***

    1) В первый кластер попадают все маршруты, для которых поездка в заданное время оптимальна;
    
    2) Во второй кластер попадают маршруты с нежелательным временем поездки;
    
    3) В третий кластер попадают маршруты, от поездок по которым следует воздержаться в заданный период времени.

***В таком случае нам нужны данные из столбцов, хранящиеся в таблице "taxi_trips":***

    1) trip_distance;

    2) lpep_dropoff_datetime;

    3) lpep_pickup_datetime.

***После того, как разобрались, какие данные нам нужны для кластеризации, можно начать разрабатывать функцию, загружающая необходимые данные порциями в целях предотвращения перегрузки системы.***

In [ ]:
def fetch_trip_data(db_path, batch_size, max_rows=1000000):
    """
    Функция загружает необходимые данные для кластеризации пачками
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    offset = 0 # Смещение
    processed_rows = 0 # Количество загруженных строк

    while processed_rows < max_rows:
        cursor.execute(f"""
            SELECT trip_distance, 
                   (julianday(lpep_dropoff_datetime) - julianday(lpep_pickup_datetime)) * 1440 AS trip_duration
            FROM taxi_trips
            WHERE trip_distance IS NOT NULL 
              AND lpep_pickup_datetime IS NOT NULL
            LIMIT {batch_size} OFFSET {offset};
        """)
        
        batch = cursor.fetchall()
        if not batch: # Если нет данных
            break
        yield batch
        """
        Увеличивает значения, чтобы продолжить загрузку
        """
        offset += batch_size
        processed_rows += len(batch)
    
    conn.close()

***Далее нужно классифицировать кластеры (проименовать) по заданным категориям. Для этого нужно написать функцию, которая возвращает классификацию (имя) кластера.***

In [5]:
def classify_label(label):
    """
    Функция делает классификацию кластеров
    """
    return "Оптимальные" if label == 0 else "Нежелательные" if label == 1 else "Нежелательные совсем"

***Ранее мы создавали функцию, отвечающая за создание таблицы для хранения кластеров. Теперь нужна функция, которая сохраняет результаты кластеризации в таблицу "trip_clusters", записывая по частям, чтобы не нагрузить систему.***

In [6]:
def save_clusters(db_path, batch, kmeans_labels, dbscan_labels, hierarchical_labels):
    """
    Функция сохраняет результаты кластеризации в таблицу пачками
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    for i in range(len(batch)):
        cursor.execute("""
            INSERT INTO trip_clusters (trip_distance, trip_duration, cluster_kmeans, cluster_dbscan, cluster_hierarchical)
            VALUES (?, ?, ?, ?, ?)
        """, (
            batch[i][0], batch[i][1], 
            classify_label(kmeans_labels[i]), 
            classify_label(dbscan_labels[i]), 
            classify_label(hierarchical_labels[i])
        ))

    conn.commit()
    conn.close()

***Далее надо свизуализиовать распределения данных по кластерам для наглядностей и ясности. Для удобства стоит написать функцию, которая делает визуализацию кластерных структур.***

In [14]:
def visualize_clusters(kmeans_labels_all, dbscan_labels_all, hierarchical_labels_all, trip_distances, trip_durations):
    """
    Функция делает визуализацию кластерных структур
    """
    plt.figure(figsize=(12, 6)) # Создаёт график нужного размера

    """
    Первый подграфик - KMeans Clustering
    """
    plt.subplot(1, 3, 1) # Задаёт номер подграфика
    plt.scatter(trip_distances, trip_durations, c=kmeans_labels_all, cmap='viridis', marker='o')
    plt.title('KMeans Clustering')
    plt.xlabel('Trip Distance')
    plt.ylabel('Trip Duration')
    plt.colorbar() # Создаёт colorbar для определения кластера
    plt.xticks([0, 100, 200, 300, 400, 500]) # Делит ось Х на значения из "Trip Distance"
    plt.subplots_adjust(wspace=0.5)

    """
    Второй подграфик - DBSCAN Clustering
    """
    plt.subplot(1, 3, 2) # Задаёт номер подграфика
    plt.scatter(trip_distances, trip_durations, c=dbscan_labels_all, cmap='plasma', marker='o')
    plt.title('DBSCAN Clustering')
    plt.xlabel('Trip Distance')
    plt.ylabel('Trip Duration')
    plt.colorbar() # Создаёт colorbar для определения кластера
    plt.xticks([0, 100, 200, 300, 400, 500]) # Делит ось Х на значения из "Trip Distance"
    plt.subplots_adjust(wspace=0.5)

    """
    Третий подграфик - AgglomerativeClustering
    """
    plt.subplot(1, 3, 3) # Задаёт номер подграфика
    plt.scatter(trip_distances, trip_durations, c=hierarchical_labels_all, cmap='inferno', marker='o')
    plt.title('AgglomerativeClustering')
    plt.xlabel('Trip Distance')
    plt.ylabel('Trip Duration')
    plt.colorbar() # Создаёт colorbar для определения кластера
    plt.xticks([0, 100, 200, 300, 400, 500]) # Делит ось Х на значения из "Trip Distance"
    plt.subplots_adjust(wspace=0.5)

    """
    Вывод графика
    """
    plt.suptitle("Кластерная структура")
    plt.show()

***Теперь, когда написали базовые функции для работы, можно приступить к применению моделей кластеризации. Нужно написать общую функцию, которая охватывает все созданные ранее базовые "инструменты", и которая будет проводить кластерный анализ:***

    1) Сначала масштабируем данные (оно гарантирует, что каждый признак вносит равный вклад в расчеты расстояния);

    2) После масштабирования данных применить модель кластеризации.

***Для кластеризации возьмём три модели:***

    1) KMeans;

    2) Agglomerative Clustering;

    3) DBSCAN.

***Под конец кластеризации функция выводит визуализацию кластерных структур, ориентируясь по дистанции и времени поездки.***

In [8]:
def apply_clustering(db_path, batch_size):
    """
    Функция проводит кластерный анализ и выводит визуализацию кластерных структур
    """
    print("=== Запуск кластеризации ===")

    create_cluster_table(db_path)

    """
    Для визуализации сохраняет необходимые данные в списки
    """
    trip_distances = []
    trip_durations = []
    kmeans_labels_all = []
    dbscan_labels_all = []
    hierarchical_labels_all = []

    scaler = StandardScaler() # Модель масштабируемости данных

    for batch in fetch_trip_data(db_path, batch_size):
        trip_distance = np.array([row[0] for row in batch]).reshape(-1, 1)
        trip_duration = np.array([row[1] for row in batch]).reshape(-1, 1)
        data = np.hstack((trip_distance, trip_duration))

        scaled_data = scaler.fit_transform(data) # Масштабирует данные

        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10) # Алгоритм (модель) кластеризации
        dbscan = DBSCAN(eps=0.5, min_samples=10) # Алгоритм (модель) кластеризации
        hierarchical = AgglomerativeClustering(n_clusters=3) # Алгоритм (модель) кластеризации

        kmeans_labels = kmeans.fit_predict(scaled_data)  # Применяет модель кластеризации
        dbscan_labels = dbscan.fit_predict(scaled_data)  # Применяет модель кластеризации
        hierarchical_labels = hierarchical.fit_predict(scaled_data)  # Применяет модель кластеризации

        """
        Сохраняет результаты в определённые списки
        """
        trip_distances.extend(trip_distance.flatten())
        trip_durations.extend(trip_duration.flatten())
        kmeans_labels_all.extend(kmeans_labels)
        dbscan_labels_all.extend(dbscan_labels)
        hierarchical_labels_all.extend(hierarchical_labels)

        """
        Сохраняет кластеры в таблицу БД
        """
        save_clusters(db_path, batch, kmeans_labels, dbscan_labels, hierarchical_labels)

    print("=== Кластеризация завершена! ===")

    """
    Выводит визуализацию кластерных структур
    """
    visualize_clusters(kmeans_labels_all, dbscan_labels_all, hierarchical_labels_all, trip_distances, trip_durations)

In [ ]:
DB_PATH = "taxi_trips.db" # Путь к БД
BATCH_SIZE = 10000 # Размер пакета данных (лучше 10 000, чтобы не перегрузить систему)

apply_clustering(DB_PATH, BATCH_SIZE)

### *2.4 Вычисление показателей производительности кластеризации*
Выберите метрику оценки качества кластеризации. Обоснуйте выбор методов и приемов. Выполните визуальный анализ кластерных структур и оценки качества кластеризации. Определите лучший алгоритм кластеризации на основе выбранной метрики.

***Также, как в предыдущем задании, нужно разработать функцию, загружающая необходимые данные порциями в целях предотвращения перегрузки системы. Однако в этот раз:***

    1) Мы загружаем данные из таблицы "trip_clusters", так как нам нужны готовые кластеры, которые мы получили после кластеризации;

    2) Кластеры переводим в числовые метки, чтобы корректно применить в вычислении оценки качества кластеризации (0 - Оптимальные, 1 - Нежелательные, 2 - Нежелательные совсем).

In [15]:
def fetch_clusters_from_db(db_path, batch_size):
    """
    Функция загружает необходимые данные для вычисления оценки пачками
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    offset = 0 # Смещение
    
    while True:
        cursor.execute(f"""
            SELECT trip_distance, 
                   trip_duration, 
                   cluster_kmeans, 
                   cluster_dbscan, 
                   cluster_hierarchical
            FROM trip_clusters
            LIMIT {batch_size} OFFSET {offset};
        """)
        
        batch = cursor.fetchall()
        if not batch: # Если нет данных
            break
        
        """
        Для визуализации создаём списки и сохраняем численные значения, а метки кластеров переводим в числа (от 0 до 2)
        """
        trip_distances = np.array([row[0] for row in batch])
        trip_durations = np.array([row[1] for row in batch])

        kmeans_labels = [0 if row[2] == 'Оптимальные' else 1 if row[2] == 'Нежелательные' else 2 for row in batch]
        dbscan_labels = [0 if row[3] == 'Оптимальные' else 1 if row[3] == 'Нежелательные' else 2 for row in batch]
        hierarchical_labels = [0 if row[4] == 'Оптимальные' else 1 if row[4] == 'Нежелательные' else 2 for row in batch]

        yield trip_distances, trip_durations, kmeans_labels, dbscan_labels, hierarchical_labels
        
        offset += batch_size # Увеличивает смещение, чтобы продолжить загрузку

    conn.close()

***Для оценки качества кластеризации можно использовать метрику "Silhouette Score", так как данная метрика оценивает, насколько объекты внутри кластеров похожи друг на друга, и насколько различаются между кластерами (значение от -1 до 1, и чем значение ближе к 1, тем лучше). Для удобства напишем функцию, которая:***

    1) Вычисляет оценку качества кластеризации, а также даёт пояснение по поводу выбора метрики;

    2) Делает сравнительный анализ в текстовом и визуальном виде;

    3) Определяет лучший алгоритм кластеризации.

In [16]:
def evaluate_clustering_batchwise(db_path, batch_size=10000):
    """
    Функция вычисляет оценки качества кластеризации и, делая сравнительный анализ, определяет лучший алгоритм кластеризации
    """
    print("=== Оценка качества кластеризации ===\n")

    """
    Для визуализации сохраняет необходимые данные в списки
    """
    kmeans_labels_all = []
    dbscan_labels_all = []
    hierarchical_labels_all = []
    trip_distances_all = []
    trip_durations_all = []

    """
    Для вычисления средней Silhouette Score сохраняет общую оценку
    """
    kmeans_silhouette_total = 0
    dbscan_silhouette_total = 0
    hierarchical_silhouette_total = 0

    batch_count = 0

    for trip_distances, trip_durations, kmeans_labels, dbscan_labels, hierarchical_labels in fetch_clusters_from_db(db_path, batch_size):
        """
        Сохраняет переданные аргументы в определённые списки
        """
        trip_distances_all.extend(trip_distances)
        trip_durations_all.extend(trip_durations)
        kmeans_labels_all.extend(kmeans_labels)
        dbscan_labels_all.extend(dbscan_labels)
        hierarchical_labels_all.extend(hierarchical_labels)
        
        """
        Вычисляет оценки
        """
        kmeans_silhouette_batch = silhouette_score(np.column_stack([trip_distances, trip_durations]), kmeans_labels)
        dbscan_silhouette_batch = silhouette_score(np.column_stack([trip_distances, trip_durations]), dbscan_labels)
        hierarchical_silhouette_batch = silhouette_score(np.column_stack([trip_distances, trip_durations]), hierarchical_labels)

        kmeans_silhouette_total += kmeans_silhouette_batch
        dbscan_silhouette_total += dbscan_silhouette_batch
        hierarchical_silhouette_total += hierarchical_silhouette_batch

        batch_count += 1

    """
    Вычисляет среднее Silhouette Score
    """
    kmeans_silhouette_avg = kmeans_silhouette_total / batch_count
    dbscan_silhouette_avg = dbscan_silhouette_total / batch_count
    hierarchical_silhouette_avg = hierarchical_silhouette_total / batch_count

    """
    Выводит результат в текстовом виде
    """
    print(f"Silhouette Score для KMeans: {kmeans_silhouette_avg:.3f}")
    print(f"Silhouette Score для DBSCAN: {dbscan_silhouette_avg:.3f}")
    print(f"Silhouette Score для Agglomerative Clustering: {hierarchical_silhouette_avg:.3f}")

    """
    Выводит обоснование выбора метрики
    """
    print("\nОбоснование выбора метрики:")
    print("Метрика \"Silhouette Score\" оценивает, насколько объекты внутри кластеров похожи друг на друга, и насколько различаются между кластерами.\nЗначение от -1 до 1, и чем значение ближе к 1, тем лучше.")

    plt.figure(figsize=(12, 6)) # Создаёт график нужного размера

    """
    Определяет необходимые данные для построения графика
    """
    algorithms = ['KMeans', 'DBSCAN', 'Agglomerative Clustering']
    scores = [kmeans_silhouette_avg, dbscan_silhouette_avg, hierarchical_silhouette_avg]

    """
    Создаёт визуализацию
    """
    plt.bar(algorithms, scores, color=['skyblue', 'salmon', 'lightgreen'])
    plt.title("Сравнение Silhouette Score для разных алгоритмов кластеризации")
    plt.xlabel('Алгоритмы кластеризации')
    plt.ylabel('Silhouette Score')
    plt.ylim([-1, 1])
    plt.show()

    """
    Определяет лучший алгоритм кластеризации
    """
    best_algorithm = max(zip(algorithms, scores), key=lambda x: x[1])[0]
    print(f"Лучший алгоритм кластеризации: {best_algorithm}")
    print(f"Эти результаты говорят нам о том, что кластеры, выделенные алгоритмом \"{best_algorithm}\", хорошо отделены друг от друга и имеют значительное внутрикластерное сходство.")

In [ ]:
DB_PATH = "taxi_trips.db" # Путь к БД

evaluate_clustering_batchwise(DB_PATH)

### *2.5 Подготовка отчета*
Подготовьте отчет о проделанной работе по итогам модуля, в котором будут представлены результаты, выводы и обоснования выбора по каждому разделу задания. В отчете также опишите содержимое результирующих файлов

***В результате всей модули я сформировал отчёт в виде блоков записей и комментариев к коду для всех вышеперечисленных пунктов.***